## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `naive_rag.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [11]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [13]:
# LISTANDO AS PERGUNTAS A SEREM RESPONDIDAS
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [14]:
# Carregando o Modelo de LLM (Large Language Model) e os Embeddings
# O modelo de embeddings é utilizado para gerar representações vetoriais dos textos
# O modelo de linguagem (LLM) será usado para gerar respostas a partir do contexto

embeddings_model = OpenAIEmbeddings()  # Modelo de embeddings para gerar representações vetoriais

# Inicializando o modelo de linguagem GPT-3.5
# Este modelo será usado para responder perguntas com base no contexto fornecido
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # Nome do modelo de linguagem
    max_tokens=200,              # Número máximo de tokens na resposta gerada
)

In [15]:
# Carregando o PDF
# O carregador (loader) será responsável por carregar o PDF e dividir o conteúdo em páginas

pdf_path = "os-sertoes.pdf"  # Caminho para o arquivo PDF

# Inicializando o carregador de PDF
# O parâmetro 'extract_images=False' garante que imagens não sejam extraídas do PDF
loader = PyPDFLoader(pdf_path, extract_images=False)

# Carregando o conteúdo do PDF e dividindo-o em páginas
# A função 'load_and_split' divide o PDF em uma lista de páginas para processamento posterior
pages = loader.load_and_split()


In [16]:
# Separando o documento em pedaços (chunks) e carregando o banco de dados vetorial
# O divisor (splitter) divide o conteúdo do PDF em pedaços menores (chunks) para facilitar o processamento

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          # Tamanho máximo de cada pedaço (chunk)
    chunk_overlap=20,         # Sobreposição entre os pedaços (chunks)
    length_function=len,      # Função para calcular o comprimento dos pedaços
    add_start_index=True,     # Adiciona o índice de início a cada pedaço
)

# Dividindo o conteúdo do PDF (páginas) em pedaços menores (chunks)
chunks = text_splitter.split_documents(pages)

# Criando o banco de dados vetorial com os pedaços divididos
# Usamos o modelo de embeddings para converter os pedaços em vetores para armazenamento no banco de dados vetorial
db = Chroma.from_documents(chunks, embedding=embeddings_model)

In [17]:
# Configurando o recuperador (retriever) para buscar os documentos mais relevantes
# A busca é realizada com base nos embeddings armazenados no banco de dados vetorial

retriever = db.as_retriever(search_kwargs={"k": 3})  # Configurando a busca para retornar os 3 documentos mais relevantes


In [18]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [19]:
# Função para responder a uma pergunta com base no contexto recuperado
# A função usa o recuperador (retriever) para obter o contexto relevante e depois gera a resposta utilizando a sequência

def answer_question(question: str):
    # Recuperando o contexto relevante para a pergunta usando o recuperador
    context = retriever.invoke(question)

    # Gerando a resposta com o modelo de linguagem e o parser, passando o contexto e a pergunta
    response = sequence.invoke({"context": context, "question": question})

    # Retornando a resposta gerada
    return response

In [20]:
# Iterando sobre as perguntas e obtendo as respostas
# 'enumerate' permite obter tanto o índice quanto a pergunta de cada item na lista
for index, question in enumerate(questions):
    # Obtendo a resposta para a pergunta atual utilizando a função 'answer_question'
    resposta = answer_question(question)
    # Exibindo o número da pergunta, a pergunta e a resposta gerada
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como um lugar de extrema dificuldade e desafio, onde a vida é marcada pela aridez, pela falta de recursos e pela luta constante pela sobrevivência. Ele destaca a importância da terra fértil na região, que libertou os indígenas, mas também menciona a cultura extensiva da cana, importada da Madeira, que determinou o esquecimento dos sertões.\n\nEssa realidade do sertão nordestino influencia diretamente a vida dos habitantes, que são obrigados a enfrentar condições adversas, como a escassez de água, a falta de infraestrutura e a ausência de oportunidades. Euclides da Cunha mostra como essas dificuldades moldam o caráter e a cultura das pessoas que viv'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por E